# GRU

## 介绍

在class9_part2中, 发现当时间步数较大或较小的时候, RNN会出现梯度衰减/爆炸。虽然梯度裁剪可以应对梯度爆炸, 但是无法解决梯度衰减问题, 由于这个原因, RNN在实际中难以捕捉ts中时间步距离较大的依赖关系。因此, 提出了GRN(gated recurrent neural network), 它可以通过门来控制信息的流动。

在GRN中, 门控循环单元（gated recurrent unit，GRU）是一种常用的门控循环神经网络有两个门, 分别为重置门(reset gate)和更新门(update gate)。如下图所示, 门控循环单元中的重置门和更新门的输入均为当前时间步输入$\boldsymbol{X}_t$与上一时间步隐藏状态$\boldsymbol{H}_{t-1}$，输出由激活函数为sigmoid函数的全连接层计算得到。

具体来说，假设隐藏单元个数为$h$，给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$（样本数为$n$，输入个数为$d$）和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。重置门$\boldsymbol{R}_t \in \mathbb{R}^{n \times h}$和更新门$\boldsymbol{Z}_t \in \mathbb{R}^{n \times h}$的计算如下：

$$
\begin{aligned}
\boldsymbol{R}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xr} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hr} + \boldsymbol{b}_r),\\
\boldsymbol{Z}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xz} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hz} + \boldsymbol{b}_z),
\end{aligned}
$$

其中$\boldsymbol{W}_{xr}, \boldsymbol{W}_{xz} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hr}, \boldsymbol{W}_{hz} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_r, \boldsymbol{b}_z \in \mathbb{R}^{1 \times h}$是偏差参数。在MLP中介绍过，sigmoid函数可以将元素的值变换到0和1之间。因此，重置门$\boldsymbol{R}_t$和更新门$\boldsymbol{Z}_t$中每个元素的值域都是$[0, 1]$。

<img src="img/class10_1_1.1.svg" style="zoom:100%">

接下来, GRU将计算**候选隐藏状态**来辅助稍后的隐藏状态计算。如下图所示, 将当前时间步RG的输出$\boldsymbol{R}_t$与上一时间步的隐藏状态$\boldsymbol{H}_{t-1}$做元素乘法(符号为$\odot$)。如果RG中元素值接近0, 那么意味这重置对应隐藏状态元素为0, 即丢弃上一时间步的隐藏状态。如果元素值接近1, 那么意味着保留上一时间步的隐藏状态。然后, 将按元素乘法的结果与当前时间步的输入$\boldsymbol{X}_t$进行concat, 再通过含激活函数tanh的全连接层计算出**候选隐藏状态**。

具体计算公式如下式, 其中$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$是偏差参数。从该公式可以看出，重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态。而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息。因此，重置门可以用来丢弃与预测无关的历史信息。
$$\tilde{\boldsymbol{H}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \left(\boldsymbol{R}_t \odot \boldsymbol{H}_{t-1}\right) \boldsymbol{W}_{hh} + \boldsymbol{b}_h),$$

<img src="img/class10_1_1.2.svg" style="zoom:100%">

最后，计算时间步$t$的隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$。其计算使用当前时间步的更新门$\boldsymbol{Z}_t$来对上一时间步的隐藏状态$\boldsymbol{H}_{t-1}$和当前时间步的候选隐藏状态$\tilde{\boldsymbol{H}}_t$做组合：

$$\boldsymbol{H}_t = \boldsymbol{Z}_t \odot \boldsymbol{H}_{t-1}  + (1 - \boldsymbol{Z}_t) \odot \tilde{\boldsymbol{H}}_t.$$

更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，如下图所示。假设更新门在时间步$t'$到$t$（$t' < t$）之间一直近似1。那么，在时间步$t'$到$t$之间的输入信息几乎没有流入时间步$t$的隐藏状态$\boldsymbol{H}_t$。实际上，这可以看作是较早时刻的隐藏状态$\boldsymbol{H}_{t'-1}$一直通过时间保存并传递至当前时间步$t$。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

<img src="img/class10_1_1.3.svg" style="zoom:100%">

总结, 可以理解为重置门有助于捕捉时间序列里短期的依赖关系; 更新门有助于捕捉序列里长期的依赖关系。
- 候选隐藏状态中含有的信息为: 当前时间步的输入与权重之积 + 上一个时间步的隐藏状态信息(由重置门控制其信息量多寡)
- GRU输出的当前时间步隐藏状态含有的信息为: 上一个时间步的隐藏状态信息(由更新门控制其信息量多寡) + 当前时间步的候选隐藏状态信息(由更新门控制其信息量多寡; 前面一个信息越多这个信息越少)

## 从零实现

In [211]:
# 读取数据集
import tensorflow as tf
from tensorflow import keras
import time
import math
import numpy as np
import sys
sys.path.append('./Dive-into-DL-TF2.0/d2lzh_tensorflow2')
import d2lzh_tensorflow2 as d2l # 具体函数的信息全部保存在/files 内

(corpus_indices, char_to_idx, idx_to_char,vocab_size) = d2l.load_data_jay_lyrics()

In [212]:
# 初始化模型参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        return tf.Variable(tf.random.normal(shape=shape, stddev=0.01, mean=0, dtype=tf.float32))
    
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                tf.Variable(tf.zeros(num_hiddens), dtype=tf.float32))
    
    # 更新门参数
    W_xz, W_hz, b_z = _three() 
    # 重置门参数
    W_xr, W_hr, b_r = _three() 
    # 候选隐藏状态参数
    W_xh, W_hh, b_h = _three() 
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = tf.Variable(tf.zeros(num_outputs), dtype=tf.float32)
    
    # 返回
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    return params

In [213]:
# 定义模型
def init_gru_state(batch_size, num_hiddens):
    return (tf.zeros(shape=(batch_size, num_hiddens)), )

def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        X = tf.reshape(X,[-1,W_xh.shape[0]])
        Z = tf.sigmoid(tf.matmul(X, W_xz) + tf.matmul(H, W_hz) + b_z)
        R = tf.sigmoid(tf.matmul(X, W_xr) + tf.matmul(H, W_hr) + b_r)
        H_tilda = tf.tanh(tf.matmul(X, W_xh) + tf.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = tf.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [4]:
# 训练并预测
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 535.317315, time 1.75 sec
 - 分开 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店
 - 不分开 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店 店
epoch 80, perplexity 101.416973, time 1.78 sec
 - 分开                                                  
 - 不分开                                                  
epoch 120, perplexity 48.243312, time 1.82 sec
 - 分开                                                  
 - 不分开                                                  
epoch 160, perplexity 21.229451, time 1.98 sec
 - 分开 我    我                                           
 - 不分开 我     我                                          


## 简洁实现

In [5]:
# 建模
gru_layer = keras.layers.GRU(num_hiddens,time_major=True,return_sequences=True,return_state=True)
model = d2l.RNNModel(gru_layer, vocab_size)
d2l.train_and_predict_rnn_keras(model, num_hiddens, vocab_size,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 113.410639, time 1.57 sec
 - 分开  我  你  我  你  我  你  我  你  我  你  我  你  我  你  我  你  
 - 不分开  我  你  我  你  我  你  我  你  我  你  我  你  我  你  我  你  
epoch 80, perplexity 23.589051, time 1.58 sec
 - 分开 一 一一一一 一 一一一一 一 一一一一 一 一一一一 一 一一一一 一 一一一一 一 一一一一 
 - 不分开 一 一一一一 一 一一一一 一 一一一一 一 一一一一 一 一一一一 一 一一一一 一 一一一一 
epoch 120, perplexity 25.287534, time 1.54 sec
 - 分开始移动  我想要你 我不上  我不上  我不上  我不上  我不上  我不上  我不上  我不上  
 - 不分开始乡相信命就怎么跟了它在空中  我想要你 我不上  我不上  我不上  我不上  我不上  我不上 
epoch 160, perplexity 393.658532, time 1.52 sec
 - 分开球 我有一双翅  想就这样牵着你  有一双翅膀  想就这样牵着你  有一双翅膀  想就这样牵着你  
 - 不分开 我有一双翅膀  想就这样牵着你  有一双翅膀  想就这样牵着你  有一双翅膀  想就这样牵着你  


# LSTM

## 介绍

LSTM中引入了3个门, 输入门(input gate)、遗忘门(forget gate)、输出门(output gate), 以及与隐藏状态形状相同的记忆细胞, 从而记录额外的信息。

如下图所示, LSTM的门的输入均为输入均为当前时间步输入$\boldsymbol{X}_t$与上一时间步隐藏状态$\boldsymbol{H}_{t-1}$，输出由激活函数为sigmoid函数的全连接层计算得到。如此一来，这3个门元素的值域均为$[0,1]$。

具体来说，假设隐藏单元个数为$h$，给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$（样本数为$n$，输入个数为$d$）和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。
时间步$t$的输入门$\boldsymbol{I}_t \in \mathbb{R}^{n \times h}$、遗忘门$\boldsymbol{F}_t \in \mathbb{R}^{n \times h}$和输出门$\boldsymbol{O}_t \in \mathbb{R}^{n \times h}$分别计算如下：

$$
\begin{aligned}
\boldsymbol{I}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xi} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hi} + \boldsymbol{b}_i),\\
\boldsymbol{F}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xf} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hf} + \boldsymbol{b}_f),\\
\boldsymbol{O}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xo} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{ho} + \boldsymbol{b}_o),
\end{aligned}
$$

其中的$\boldsymbol{W}_{xi}, \boldsymbol{W}_{xf}, \boldsymbol{W}_{xo} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hi}, \boldsymbol{W}_{hf}, \boldsymbol{W}_{ho} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_i, \boldsymbol{b}_f, \boldsymbol{b}_o \in \mathbb{R}^{1 \times h}$是偏差参数。

<img src="img/class10_2.1_1.svg" style="zoom:100%">

接下来, LSTM需要计算候选记忆细胞$\tilde{\boldsymbol{C}}_t$。它的计算与上面介绍的3个门类似，但使用了值域在$[-1, 1]$的tanh函数作为激活函数。

具体来说，时间步$t$的候选记忆细胞$\tilde{\boldsymbol{C}}_t \in \mathbb{R}^{n \times h}$的计算为

$$
\tilde{\boldsymbol{C}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xc} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hc} + \boldsymbol{b}_c),
$$

其中$\boldsymbol{W}_{xc} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hc} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_c \in \mathbb{R}^{1 \times h}$是偏差参数。

<img src="img/class10_2.1_2.svg" style="zoom:100%">

接着, 通过元素值域在$[0, 1]$的输入门、遗忘门和输出门来控制隐藏状态中信息的流动，这一般也是通过使用按元素乘法（符号为$\odot$）来实现的。当前时间步记忆细胞$\boldsymbol{C}_t \in \mathbb{R}^{n \times h}$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动：

$$\boldsymbol{C}_t = \boldsymbol{F}_t \odot \boldsymbol{C}_{t-1} + \boldsymbol{I}_t \odot \tilde{\boldsymbol{C}}_t.$$


如下图所示，遗忘门控制上一时间步的记忆细胞$\boldsymbol{C}_{t-1}$中的信息是否传递到当前时间步，而输入门则控制当前时间步的输入$\boldsymbol{X}_t$通过候选记忆细胞$\tilde{\boldsymbol{C}}_t$如何流入当前时间步的记忆细胞。如果遗忘门一直近似1且输入门一直近似0，过去的记忆细胞将一直通过时间保存并传递至当前时间步。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

<img src="img/class10_2.1_3.svg" style="zoom:100%">

最后，通过输出门来控制从记忆细胞到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的信息的流动：

$$\boldsymbol{H}_t = \boldsymbol{O}_t \odot \text{tanh}(\boldsymbol{C}_t).$$

这里的tanh函数确保隐藏状态元素值在-1到1之间。需要注意的是，当输出门近似1时，记忆细胞信息将传递到隐藏状态供输出层使用；当输出门近似0时，记忆细胞信息只自己保留。下图展示了长短期记忆中隐藏状态的计算。

<img src="img/class10_2.1_4.svg" style="zoom:100%">

## 从零实现

In [47]:
# 读取数据集
import tensorflow as tf
from tensorflow import keras
import time
import math
import numpy as np
import sys
sys.path.append('./Dive-into-DL-TF2.0/d2lzh_tensorflow2')
import d2lzh_tensorflow2 as d2l # 具体函数的信息全部保存在/files 内

(corpus_indices, char_to_idx, idx_to_char,vocab_size) = d2l.load_data_jay_lyrics()

In [48]:
# 初始化模型参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        return tf.Variable(tf.random.normal(shape=shape,stddev=0.01,mean=0,dtype=tf.float32))
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                tf.Variable(tf.zeros(num_hiddens), dtype=tf.float32))
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = tf.Variable(tf.zeros(num_outputs), dtype=tf.float32)
    return [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q]

In [49]:
# 定义模型
def init_lstm_state(batch_size, num_hiddens):
    return (tf.zeros(shape=(batch_size, num_hiddens)), 
            tf.zeros(shape=(batch_size, num_hiddens)))

def lstm(inputs, state, params):
    W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q = params
    (H, C) = state
    outputs = []
    for X in inputs:
        X = tf.reshape(X,[-1,W_xi.shape[0]])
        I = tf.sigmoid(tf.matmul(X, W_xi) + tf.matmul(H, W_hi) + b_i)
        F = tf.sigmoid(tf.matmul(X, W_xf) + tf.matmul(H, W_hf) + b_f)
        O = tf.sigmoid(tf.matmul(X, W_xo) + tf.matmul(H, W_ho) + b_o)
        C_tilda = tf.tanh(tf.matmul(X, W_xc) + tf.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * tf.tanh(C)
        Y = tf.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [10]:
# 训练模型并预测
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 7233.012047, time 1.97 sec
 - 分开的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
 - 不分开的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch 80, perplexity 2521.757151, time 1.99 sec
 - 分开的的让的的让的的让的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让
 - 不分开的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的让的的
epoch 120, perplexity 1032.075150, time 1.97 sec
 - 分开的步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步
 - 不分开的步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步步
epoch 160, perplexity 552.197864, time 2.20 sec
 - 分开的的的的的的的 的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
 - 不分开的的的的 我的的的 的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的


## 简洁实现

不能直接引用d2l里面的函数, 因为有错误, 具体出错的地方在`RNNModel`.`call`部分的state部分, 因为LSTM会输出output, H, C 三个部分, 而d2l的代码只取了output, state。然而LSTM的state除了隐藏状态之外，还包括记忆细胞的信息。

In [190]:
class RNNModel(keras.layers.Layer):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.dense = keras.layers.Dense(vocab_size)

    def call(self, inputs, state):
        # 将输入转置成(num_steps, batch_size)后获取one-hot向量表示
        X = tf.one_hot(tf.transpose(inputs), self.vocab_size)
        # 与D2L的代码主要差别就是下面两句！！！！！！！！！！！！！！！
        Y, H, C = self.rnn(X, state)
        state = (H, C)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(tf.reshape(Y,(-1, Y.shape[-1])))
        return output, state

    def get_initial_state(self, *args, **kwargs):
        return self.rnn.cell.get_initial_state(*args, **kwargs)

In [191]:
def grad_clipping(grads,theta):
    norm = np.array([0])
    for i in range(len(grads)):
        norm+=tf.math.reduce_sum(grads[i] ** 2)
    #print("norm",norm)
    norm = np.sqrt(norm).item()
    new_gradient=[]
    if norm > theta:
        for grad in grads:
            new_gradient.append(grad * theta / norm)
    else:
        for grad in grads:
            new_gradient.append(grad)  
    #print("new_gradient",new_gradient)
    return new_gradient

In [192]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = np.array(corpus_indices)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].reshape((
        batch_size, batch_len))
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [193]:
def predict_rnn_keras(prefix, num_chars, model, vocab_size, idx_to_char,
                      char_to_idx):
    # 使用model的成员函数来初始化隐藏状态
    state = model.get_initial_state(batch_size=1,dtype=tf.float32)
    output = [char_to_idx[prefix[0]]]
    #print("output:",output)
    for t in range(num_chars + len(prefix) - 1):
        X = np.array([output[-1]]).reshape((1, 1))
        Y, state = model(X, state)  # 前向计算不需要传入模型参数
        #print("state:",state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
            #print(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(np.array(tf.argmax(Y,axis=-1))))
            #print(int(np.array(tf.argmax(Y[0],axis=-1))))
    return ''.join([idx_to_char[i] for i in output])

In [198]:
def train_and_predict_rnn_keras(model, num_hiddens, vocab_size, 
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer=tf.keras.optimizers.SGD(learning_rate=lr)
    
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(
            corpus_indices, batch_size, num_steps)
        state = model.get_initial_state(batch_size=batch_size,dtype=tf.float32)
        for X, Y in data_iter:
            with tf.GradientTape(persistent=True) as tape:
                outputs, state = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(y,outputs)
            
            grads = tape.gradient(l, model.variables)
            # 梯度裁剪
            grads = grad_clipping(grads, clipping_theta)
            optimizer.apply_gradients(zip(grads, model.variables))  # 因为已经误差取过均值，梯度不用再做平均
            l_sum += np.array(l).item() * len(y)
            n += len(y)

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_keras(
                    prefix, pred_len, model, vocab_size,  idx_to_char,
                    char_to_idx))
    return model

In [199]:
lr = 1e-2
lstm_layer = keras.layers.LSTM(num_hiddens,time_major=True,return_sequences=True,return_state=True)
model = RNNModel(lstm_layer, vocab_size)
model = train_and_predict_rnn_keras(model, num_hiddens, vocab_size, 
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 41428.896290, time 1.62 sec
 - 分开担能老  葛练练居伦抬牧书B便便公妥秒唱疯疯懂我自自自忘疼 汉 汉比嘴图图旋旋撒?藤嘟嘟法办个掉丘丘
 - 不分开带  葛载纪箱信汉图图图旋旋酱篮酱替写碗笔谷髅 壶 壶面沙于人攻攻形单武篮问问琴杨度馆壶壶层令倦  
epoch 80, perplexity 22376.223492, time 1.62 sec
 - 分开  葛载练练居敢里习  葛练练居译誓生叫译誓望米我我凶凶坟能能迹城  守 弄葛练练居伦抬牧书宇的我 
 - 不分开   背背练背练敢里里  葛练练居译誓弄叫米坏我  葛练居怪横除棒棒袭袭一埋一忆能能哀简  候其哼望
epoch 120, perplexity 14734.273999, time 1.76 sec
 - 分开   葛载练的射射载的我我   汉 葛载弄葛练旋旋然一然悄壁里   葛练练居伦译胖我   汉守 弄葛
 - 不分开    葛载弄弄葛硬句谷怪髅   壶 壶 壶 壶面  壶 壶面  壶面使使使息了了了器器器器泛   
epoch 160, perplexity 11147.922867, time 1.64 sec
 - 分开    葛载弄的我   汉  弄葛载弄葛了了器器器器泛    葛檐  葛练檐我    汉弄弄葛葛硬硬
 - 不分开     葛载弄弄葛了了器器器   葛葛练檐星     葛载弄弄葛了了器器器   葛葛练檐星    


# 深度循环神经网络

下图为一个多层的RNN网络, 每个隐藏状态不断传递至当前层的下一时间步和当前时间步的下一层。

具体来说，在时间步$t$里，设小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$（样本数为$n$，输入个数为$d$），第$\ell$隐藏层（$\ell=1,\ldots,L$）的隐藏状态为$\boldsymbol{H}_t^{(\ell)}  \in \mathbb{R}^{n \times h}$（隐藏单元个数为$h$），输出层变量为$\boldsymbol{O}_t \in \mathbb{R}^{n \times q}$（输出个数为$q$），且隐藏层的激活函数为$\phi$。第1隐藏层的隐藏状态和之前的计算一样：

$$\boldsymbol{H}_t^{(1)} = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(1)} + \boldsymbol{H}_{t-1}^{(1)} \boldsymbol{W}_{hh}^{(1)}  + \boldsymbol{b}_h^{(1)}),$$


其中权重$\boldsymbol{W}_{xh}^{(1)} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh}^{(1)} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h^{(1)} \in \mathbb{R}^{1 \times h}$分别为第1隐藏层的模型参数。

当$1 < \ell \leq L$时，第$\ell$隐藏层的隐藏状态的表达式为

$$\boldsymbol{H}_t^{(\ell)} = \phi(\boldsymbol{H}_t^{(\ell-1)} \boldsymbol{W}_{xh}^{(\ell)} + \boldsymbol{H}_{t-1}^{(\ell)} \boldsymbol{W}_{hh}^{(\ell)}  + \boldsymbol{b}_h^{(\ell)}),$$


其中权重$\boldsymbol{W}_{xh}^{(\ell)} \in \mathbb{R}^{h \times h}$、$\boldsymbol{W}_{hh}^{(\ell)} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h^{(\ell)} \in \mathbb{R}^{1 \times h}$分别为第$\ell$隐藏层的模型参数。

最终，输出层的输出只需基于第$L$隐藏层的隐藏状态：

$$\boldsymbol{O}_t = \boldsymbol{H}_t^{(L)} \boldsymbol{W}_{hq} + \boldsymbol{b}_q,$$

其中权重$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$和偏差$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$为输出层的模型参数。

同多层感知机一样，隐藏层个数$L$和隐藏单元个数$h$都是超参数。此外，如果将隐藏状态的计算换成门控循环单元或者长短期记忆的计算，我们可以得到深度门控循环神经网络。

<img src="img/class10_3.svg" style="zoom:100%">

# 双向循环神经网络

之前介绍的循环神经网络模型都是假设当前时间步是由前面的较早时间步的序列决定的，因此它们都将信息通过隐藏状态从前往后传递。有时候，当前时间步也可能由后面时间步决定。例如，当我们写下一个句子时，可能会根据句子后面的词来修改句子前面的用词。双向循环神经网络通过增加从后往前传递信息的隐藏层来更灵活地处理这类信息。下图即一个含单层神经元的双向循环神经网络架构。

给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$（样本数为$n$，输入个数为$d$）和隐藏层激活函数为$\phi$。在双向循环神经网络的架构中，
设该时间步正向隐藏状态为$\overrightarrow{\boldsymbol{H}}_t  \in \mathbb{R}^{n \times h}$（正向隐藏单元个数为$h$），
反向隐藏状态为$\overleftarrow{\boldsymbol{H}}_t  \in \mathbb{R}^{n \times h}$（反向隐藏单元个数为$h$）。我们可以分别计算正向隐藏状态和反向隐藏状态：

$$
\begin{aligned}
\overrightarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(f)} + \overrightarrow{\boldsymbol{H}}_{t-1} \boldsymbol{W}_{hh}^{(f)}  + \boldsymbol{b}_h^{(f)}),\\
\overleftarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(b)} + \overleftarrow{\boldsymbol{H}}_{t+1} \boldsymbol{W}_{hh}^{(b)}  + \boldsymbol{b}_h^{(b)}),
\end{aligned}
$$

其中权重$\boldsymbol{W}_{xh}^{(f)} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh}^{(f)} \in \mathbb{R}^{h \times h}$、$\boldsymbol{W}_{xh}^{(b)} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh}^{(b)} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h^{(f)} \in \mathbb{R}^{1 \times h}$、$\boldsymbol{b}_h^{(b)} \in \mathbb{R}^{1 \times h}$均为模型参数。

然后我们连结两个方向的隐藏状态$\overrightarrow{\boldsymbol{H}}_t$和$\overleftarrow{\boldsymbol{H}}_t$来得到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times 2h}$，并将其输入到输出层。输出层计算输出$\boldsymbol{O}_t \in \mathbb{R}^{n \times q}$（输出个数为$q$）：

$$\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q,$$

其中权重$\boldsymbol{W}_{hq} \in \mathbb{R}^{2h \times q}$和偏差$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$为输出层的模型参数。不同方向上的隐藏单元个数也可以不同。

<img src="img/class10_4.svg" style="zoom:100%">

# 重点总结

- GRU
    - 更新门、重置门因为经过了sigmoid函数, 所以他们的元素的值域为[0,1]
    - 重置门控制了上一个时间步的隐藏状态$\boldsymbol{H}_{t-1}$中的信息有多少传递本时间步的候选隐藏状态$\tilde{\boldsymbol{H}}_t$
    - 更新门控制了上一个时间步的隐藏状态$\boldsymbol{H}_{t-1}$与本时间步的候选隐藏状态$\tilde{\boldsymbol{H}}_t$, 如何进行权重组合, 形成本时间步的隐藏状态
    - RNN中通过梯度裁剪解决了梯度爆炸的问题, 但没解决梯度衰减。因此RNN在实际中难以捕捉ts中时间步距离较大的依赖关系。GRU通过门来控制信息流动, 来解决这个问题。

- LSTM
    - 输入门、遗忘门、输出门因为经过了sigmoid函数, 所以他们的元素的值域为[0,1]
    - 候选记忆细胞$\tilde{\boldsymbol{C}}_t$由上一时间步的隐藏状态$\boldsymbol{H}_{t-1}$和当前时间步的输入$\boldsymbol{X}_t$的一个函数生成
    - 遗忘门控制上一时间步的记忆细胞$\boldsymbol{C}_{t-1}$有多少信息进入当前时间步的记忆细胞$\boldsymbol{C}_{t}$
    - 输入门控制当前时间步的候选记忆细胞$\tilde{\boldsymbol{C}}_t$有多少信息进入当前时间步的记忆细胞$\boldsymbol{C}_{t}$
    - 输出门控制当前时间步的记忆细胞$\boldsymbol{C}_{t}$在tanh之后有多少进入当前时间步的隐藏状态$\boldsymbol{H}_t$
    - 相较于RNN与GRU, LSTM需要初始化的除了$\boldsymbol{H}_0$之外还有$\boldsymbol{C}_{0}$
    - 如果遗忘门的元素一直近似于1且输入门的元素一直近似于0, 则过去的记忆细胞一直通过时间保存并传递至当前时间步, 该设计应对了RNN中梯度衰减的问题，并更好地捕捉了TS中时间步距离较大的依赖关系

- 感觉上述的LSTM的构造还是不够简便, 不应该这样构造, 然后预测的方法里每次预测都只输入一个字符, 且输入的state都是initial的, 难道不应该模型自己能训练出一个初始state么。感觉是有问题的。
    - 解释: 因为模型训练的参数, 而H0其实是一个状态而不是参数，所以一般情况下我们都是把H0用0来初始化。
    - d2l里面的预测函数写的估计有问题, 每次都是输入一个字输出后一个字, 而不是输入一个字符串, 输入后一个字符(相当于输入只有一个时间步)